In [1]:
import xarray as xr

ds = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2022-1h-360x181_equiangular_with_poles_conservative.zarr")
ds


/home/nld4584/.local/lib/python3.11/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'earthkit' loading failed:
No module named 'earthkit.data.utils.xarray'
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)


<xarray.Dataset> Size: 34TB
Dimensions:                                           (time: 552264,
                                                       longitude: 360,
                                                       latitude: 181, level: 37)
Coordinates:
  * latitude                                          (latitude) float64 1kB ...
  * level                                             (level) int64 296B 1 .....
  * longitude                                         (longitude) float64 3kB ...
  * time                                              (time) datetime64[ns] 4MB ...
Data variables: (12/31)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 144GB dask.array<chunksize=(8, 360, 181), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 144GB dask.array<chunksize=(8, 360, 181), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 144GB dask.array<chunksize=(8, 360, 181), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 261kB dask.array<chunksize=(360, 181), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 261kB dask.array<chunksize=(360, 181), meta=np.ndarray>
    geopotential                                      (time, level, longitude, latitude) float32 5TB dask.array<chunksize=(8, 37, 360, 181), meta=np.ndarray>
    ...                                                ...
    total_precipitation                               (time, longitude, latitude) float32 144GB dask.array<chunksize=(8, 360, 181), meta=np.ndarray>
    type_of_high_vegetation                           (longitude, latitude) float32 261kB dask.array<chunksize=(360, 181), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 261kB dask.array<chunksize=(360, 181), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 5TB dask.array<chunksize=(8, 37, 360, 181), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 5TB dask.array<chunksize=(8, 37, 360, 181), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 5TB dask.array<chunksize=(8, 37, 360, 181), meta=np.ndarray>

In [1]:
import xarray as xr

ds = xr.open_zarr("/home/mlx/ai-ml/datasets//aifs-ea-an-oper-0001-mars-o96-1979-2023-6h-v8.zarr")

/home/nld4584/.local/lib/python3.11/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'earthkit' loading failed:
No module named 'earthkit.data.utils.xarray'
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)
/etc/ecmwf/ssd/ssd1/tmpdirs/nld4584.2982111.20251114_110524.088/ipykernel_2999846/1768682505.py:3: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds = xr.open_zarr("/home/mlx/ai-ml/datasets//aifs-ea-an-oper-0001-mars-o96-1979-2023-6h-v8.zarr")


In [7]:
import yaml

config = {}

# first, we specify the checkpoint to use
config["checkpoint"] = str(CHECKPOINT_PATH)

# as input, we use the test split of the dataset used for training
# 'test' is a registered input type, just like 'grib' or 'mars'
config["input"] = {
    "test": {
        "use_original_paths": True,
    }
}

# as output, let's start by simply printing to the console
config["output"] = "printer"

# finally, we specify an initialization date for the forecasts
# it must be part of the test hold-out period
config["date"] = "2022-09-01T00:00"



config_dir = PROJECT_ROOT / "configs/inference"
config_dir.mkdir(parents=True, exist_ok=True)
config_path = config_dir / "dataset_to_printer.yaml"
with open(config_path, "w") as f:
    yaml.dump(config, f)

In [8]:
import datetime
from collections import defaultdict

import numpy as np
import earthkit.data as ekd
import earthkit.regrid as ekr

#from anemoi.inference.runners.simple import SimpleRunner
from anemoi.inference.outputs.printer import print_state

from ecmwf.opendata import Client as OpendataClient

In [9]:
PARAM_SFC = ["10u", "10v", "2d", "2t", "msl", "skt", "sp", "tcw", "lsm", "z", "slor", "sdor"]
PARAM_SOIL =["vsw","sot"]
PARAM_PL = ["gh", "t", "u", "v", "w", "q"]
LEVELS = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50]
SOIL_LEVELS = [1,2]

In [10]:
DATE = OpendataClient().latest()

In [11]:
print("Initial date is", DATE)

Initial date is 2025-11-17 00:00:00


In [12]:
def get_open_data(param, levelist=[]):
    fields = defaultdict(list)
    # Get the data for the current date and the previous date
    for date in [DATE - datetime.timedelta(hours=6), DATE]:
        data = ekd.from_source("ecmwf-open-data", date=date, param=param, levelist=levelist)
        for f in data:
            # Open data is between -180 and 180, we need to shift it to 0-360
            assert f.to_numpy().shape == (721,1440)
            values = np.roll(f.to_numpy(), -f.shape[1] // 2, axis=1)
            # Interpolate the data to from 0.25 to N320
            values = ekr.interpolate(values, {"grid": (0.25, 0.25)}, {"grid": "O96"})
            # Add the values to the list
            name = f"{f.metadata('param')}_{f.metadata('levelist')}" if levelist else f.metadata("param")
            fields[name].append(values)

    # Create a single matrix for each parameter
    for param, values in fields.items():
        fields[param] = np.stack(values)

    return fields

In [13]:
fields = {}
fields.update(get_open_data(param=PARAM_SFC))
soil=get_open_data(param=PARAM_SOIL,levelist=SOIL_LEVELS)

Recovering from HTTP error [429 Too Many Requests], attempt 1 of 500
Retrying in 120 seconds
Recovering from HTTP error [429 Too Many Requests], attempt 2 of 500
Retrying in 120 seconds


<multiple>:   0%|          | 0.00/7.35M [00:00<?, ?B/s]

0d7ba9f58201cd87057fd56152365d2fd2b446a1095e8a90ddfebd367fe6f89a.npz:   0%|          | 0.00/618k [00:00<?, ?B/…

<multiple>:   0%|          | 0.00/7.33M [00:00<?, ?B/s]

20251116180000-0h-scda-fc.grib2:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

20251117000000-0h-oper-fc.grib2:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

In [14]:
mapping = {'sot_1': 'stl1', 'sot_2': 'stl2',
           'vsw_1': 'swvl1','vsw_2': 'swvl2'}
for k,v in soil.items():
    fields[mapping[k]]=v

In [15]:
fields.update(get_open_data(param=PARAM_PL, levelist=LEVELS))

<multiple>:   0%|          | 0.00/54.7M [00:00<?, ?B/s]

<multiple>:   0%|          | 0.00/54.9M [00:00<?, ?B/s]

In [16]:
# Transform GH to Z
for level in LEVELS:
    gh = fields.pop(f"gh_{level}")
    fields[f"z_{level}"] = gh * 9.80665

In [ ]:
#Create Initial State
input_state = dict(date=DATE, fields=fields)
input_state


{'10u': array([[-0.42897448, -2.374193  , -4.11758737, ..., -3.7198077 ,
         -4.14512278, -4.01863398],
        [ 0.78085699, -1.16650076, -3.09441753, ..., -3.69095718,
         -3.88552528, -3.52977831]]),
 '10v': array([[-5.47139662, -5.63024832, -4.75394747, ...,  0.65043783,
         -0.72521259, -2.20744837],
        [-5.3655695 , -5.54367422, -5.00944662, ...,  0.33113783,
         -0.87626562, -2.2335014 ]]),
 '2d': array([[248.59289608, 248.68236763, 248.74914608, ..., 231.80606678,
         231.86856678, 231.90409523],
        [248.80076599, 249.06490219, 249.04648754, ..., 230.99254444,
         230.97412979, 230.98268669]]),
 '2t': array([[250.80890278, 250.93818123, 250.93818123, ..., 235.91846573,
         236.06615883, 236.10596573],
        [250.83475059, 251.07191524, 251.18407989, ..., 235.01239285,
         235.05219975, 235.06075665]]),
 'msl': array([[103194.42570805, 103153.16391609, 103126.25919931, ...,
         100602.04457517, 100575.13985839, 100555.1398

In [2]:
import earthkit.data
print(earthkit.data.__version__)
#print(anemoi.inference.__version__)

0.16.0


In [1]:
import anemoi.inference
print(anemoi.inference.__version__)

0.7.0


In [5]:
CHECKPOINT_PATH = "/ec/res4/hpcperm/ecme5801/louca/inference-last.ckpt"
!anemoi-inference inspect --variables {CHECKPOINT_PATH} 

usage: anemoi-inference [-h] [--version] [--debug]
                        {couple,inspect,metadata,patch,requests,retrieve,run,sanitise,validate}
                        ...
anemoi-inference: error: unrecognized arguments: --variables


In [6]:
!anemoi-inference inspect --datasets {CHECKPOINT_PATH} 

usage: anemoi-inference [-h] [--version] [--debug]
                        {couple,inspect,metadata,patch,requests,retrieve,run,sanitise,validate}
                        ...
anemoi-inference: error: unrecognized arguments: --datasets
